In [2]:
import re
import numpy as np
import pandas as pd
from keras.models import *
from keras.layers import *
from keras.layers import *
from keras import optimizers
from keras.utils import np_utils
from keras.callbacks import TensorBoard
from keras.models import load_model
import math
import os.path


Using TensorFlow backend.


/Users/Zizy/anaconda/envs/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [3]:
reg = "(.*) (.*) (.*)\@(.*)\^(.*)\-(.*)\+(.*)\=(.*)\_(.*)\%(.*)\^(.*)\_(.*)\~(.*)\-(.*)\!(.*)\[(.*)\$(.*)\](.*)/A:(.*)\-(.*)\-(.*)\@(.*)\~(.*)/B:(.*)\_(.*)\_(.*)\@(.*)\|(.*)/C:(.*)\+(.*)\+(.*)\@(.*)\&(.*)/D:(.*)\!(.*)\#(.*)\$(.*)\%(.*)\|(.*)\&(.*)\;(.*)\-(.*)/E:(.*)\](.*)\^(.*)\=(.*)\~(.*)\!(.*)\@(.*)\#(.*)\+(.*)\](.*)\$(.*)\|(.*)\[(.*)\&(.*)\](.*)\=(.*)\^(.*)\~(.*)\#(.*)\_(.*)\;(.*)\$(.*)\&(.*)\%(.*)\[(.*)\|(.*)\](.*)\-(.*)\^(.*)\+(.*)\~(.*)\=(.*)\@(.*)\$(.*)\!(.*)\%(.*)\#(.*)\|(.*)\|(.*)\-(.*)\&(.*)\&(.*)\+(.*)\[(.*)\;(.*)\](.*)\;(.*)\~(.*)\~(.*)\^(.*)\^(.*)\@(.*)\[(.*)\#(.*)\=(.*)\!(.*)\~(.*)\+(.*)\!(.*)\^(.*)/F:(.*)\#(.*)\#(.*)\-(.*)\$(.*)\$(.*)\+(.*)\%(.*)\;(.*)/G:(.*)\_(.*)/H:(.*)\_(.*)/I:(.*)\_(.*)/J:(.*)\~(.*)\@(.*)"
li = re.findall(reg,"0 12000000 p@xx^xx-pau+d=e_xx%xx^00_00~00-1!1[xx$xx]xx/A:xx-xx-xx@xx~xx/B:1_1_1@xx|xx/C:2+1+1@JPN&0/D:xx!xx#xx$xx%xx|xx&xx;xx-xx/E:xx]xx^2=2/4~100!1@120#48+xx]1$1|0[12&0]48=0^100~xx#xx_xx;xx$xx&xx%xx[xx|0]0-n^xx+xx~xx=xx@xx$xx!xx%xx#xx|xx|xx-xx&xx&xx+xx[xx;xx]xx;xx~xx~xx^xx^xx@xx[xx#xx=xx!xx~xx+xx!xx^xx/F:A4#7#2-2/4$100$1+45%18;xx/G:xx_xx/H:xx_xx/I:13_13/J:3~3@6")

phonemes = ['pau','xx'] +["ny","ty","py","ky","ry","f","br","sil","cl","a","i","u","e","o","k","g","s","z","sh","j","t","d","ch","q","ts","h","b","p","m","y","r","w","N","n","v" ]
pitches = ['xx']+[pitch + str(i) for i in range(1,8) for pitch in ["C", "Db", "D", "Eb", "E", "F", "Gb", "G", "Ab", "A", "Bb", "B"]] 

local_path='/Users/Zizy/Programming/HKU/Dissertation/'
lbl_name = ['t0','t1']+['p' + str(i) for i in range(1, 17)] + \
           ['a' + str(i) for i in range(1, 6)] + \
           ['b' + str(i) for i in range(1, 6)] + \
           ['c' + str(i) for i in range(1, 6)] + \
           ['d' + str(i) for i in range(1, 10)] + \
           ['e' + str(i) for i in range(1, 61)] + \
           ['f' + str(i) for i in range(1, 10)] + \
           ['g' + str(i) for i in range(1, 3)] + \
           ['h' + str(i) for i in range(1, 3)] + \
           ['i' + str(i) for i in range(1, 3)] + \
           ['j' + str(i) for i in range(1, 4)] 


In [4]:
def read_songs_label_by_file():
    labs = []
    for i in range(0, 71):
        label_file_name = 'nitech_jp_song070_f001_0' + (('0' + str(i)) if i < 10 else str(i)) + '.lab'
        if not os.path.isfile(local_path + 'lab/' + label_file_name):
            continue
        with open(local_path + 'lab/' + label_file_name, 'r') as lab_file:
            lines = lab_file.readlines()
            params_list = []
            for line in lines:
                ps = re.findall(reg, line)
                params_list.append(ps[0])
            temp = []
            for params in params_list:
                params_temp = {}
                for i in range(0, 120):
                    params_temp[lbl_name[i]] = params[i]
                temp.append(params_temp)
            params_list = temp

            labs.append(params_list)
    return labs





def read_songs_labels():
    labs = None
    for i in range(0, 71):
        label_file_name = 'nitech_jp_song070_f001_0' + (('0' + str(i)) if i < 10 else str(i)) + '.lab'
        if not os.path.isfile(local_path + 'lab/' + label_file_name):
            continue
        with open(local_path + 'lab/' + label_file_name, 'r') as lab_file:
            lines = lab_file.readlines()
            params_list = []
            for line in lines:
                ps = re.findall(reg, line)
                params_list.append(ps[0])
            temp = []
            for params in params_list:
                params_temp = {}
                for i in range(0, 120):
                    params_temp[lbl_name[i]] = params[i]
                temp.append(params_temp)
            params_list = temp
            if labs is None:
                labs = np.array(params_list)
            else:
                labs = np.append(labs, np.array(params_list))
    return labs


def make_one_hot(data1, size):
    data1 = np.array(data1)
    return (np.arange(size) == data1[:, None]).astype(np.integer)


def make_class(data, classes):
    return [classes.index(p) for p in data]

def get_params_by_name(name, params_list):
    return [params[name] for params in params_list]


def convert_params_to_one_hot(name, classes, params_list):
    data = get_params_by_name(name, params_list)
    data = make_class(data, classes)
    data = make_one_hot(data, len(classes))
    return data


# with mono file updated
def read_songs_labels_with_mono():
    labs = []
    for i in range(0, 71):
        label_file_name = 'nitech_jp_song070_f001_0' + (('0' + str(i)) if i < 10 else str(i)) + '.lab'
        if not os.path.isfile(local_path + 'lab/' + label_file_name):
            continue
        if not os.path.isfile(local_path + 'mono/' + label_file_name):
            continue

        lab_file = open(local_path + 'lab/' + label_file_name, 'r')
        lab_file_mono = open(local_path + 'mono/' + label_file_name, 'r')
        lines = lab_file.readlines()
        lines_mono = lab_file_mono.readlines()
        params_list = []
        for index, line in enumerate(lines):
            ps = list(re.findall(reg, line)[0])
            ps2 = re.findall("(.*) (.*) (.*)", lines_mono[index])[0]

            ps[0] = ps2[0]
            ps[1] = ps2[1]
            params_list.append(ps)
        temp = []
        for params in params_list:
            params_temp = {}
            for i in range(0, 120):
                params_temp[lbl_name[i]] = params[i]
            temp.append(params_temp)
        params_list = temp

        lab_file.close()
        lab_file_mono.close()

        labs.append(params_list)
    return labs


def timing_extract(labs):
    
    
    temp = []
    stacked_timing = np.array([])
    for lab in labs:
        temp.append(len(lab))
    max_sample_size = max(temp)
    print("max sample size: %s" % max_sample_size)

    for lab in labs:
        t0 = np.array(get_params_by_name('t0', lab)).astype(float) / 10000
        t1 = np.array(get_params_by_name('t1', lab)).astype(float) / 10000
        timing = t1 - t0
        if len(stacked_timing) == 0:
            stacked_timing = np.array([timing])
        else:
            stacked_timing =np.append(stacked_timing,timing)
    return stacked_timing


def feature_extract(lab):
    t0 = np.array(get_params_by_name('t0', lab)).astype(float) / 10000
    t1 = np.array(get_params_by_name('t1', lab)).astype(float) / 10000
    params_phoneme_duration = t1 - t0

    params_phoneme_duration_pre = np.insert(params_phoneme_duration[:-1], 0, np.average(params_phoneme_duration))
    params_phoneme_duration_next = np.append(params_phoneme_duration[1:], np.average(params_phoneme_duration))
    params_phonemes_one_hot = convert_params_to_one_hot('p4', phonemes, lab)
    params_phonemes_pre_one_hot = convert_params_to_one_hot('p3', phonemes, lab)
    params_phonemes_next_one_hot = convert_params_to_one_hot('p5', phonemes, lab)
    params_pitches_one_hot = convert_params_to_one_hot('e1', pitches, lab)
    params_pitches_pre_one_hot = convert_params_to_one_hot('d1', pitches, lab)
    params_pitches_next_one_hot = convert_params_to_one_hot('f1', pitches, lab)

    stacked_feature = np.hstack([params_phonemes_one_hot,
                                 params_phonemes_pre_one_hot,
                                 params_phonemes_next_one_hot,
                                 params_pitches_one_hot,
                                 params_pitches_pre_one_hot,
                                 params_pitches_next_one_hot,
                                 np.reshape(t0, (len(t0), 1)),
                                 np.reshape(t1, (len(t1), 1)),
                                 np.reshape(params_phoneme_duration, (len(params_phoneme_duration), 1)),
                                 np.reshape(params_phoneme_duration_pre, (len(params_phoneme_duration_pre), 1)),
                                 np.reshape(params_phoneme_duration_next, (len(params_phoneme_duration_next), 1)),
                                 ])
    return stacked_feature


In [5]:
# X
labs = np.array(read_songs_labels())
X = feature_extract(labs)
# X = timing_extract_x(labs)

# Y
labs_y = read_songs_labels_with_mono()
Y = timing_extract(labs_y)


max sample size: 577


45.0

In [2]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import TensorBoard

# This returns a tensor
inputs = Input(shape=(371,))

# a layer instance is callable on a tensor, and returns a tensor
# x = Dense(64, activation='relu')(inputs)
# x = Dense(32, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# predictions = Dense(2, activation='linear')(x)

model0 = load_model(local_path + 'best.h5')
x = model0.output
predictions = Dense(1, activation='linear')(x)
model = Model(inputs=model0.input, output=predictions)
# predictions = Dense(1, activation='linear')(x)

# This creates a model that includes
# the Input layer and three Dense layers
# model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=Adam(),
              loss='mse',
              )
model.summary()
model.fit(X, Y, nb_epoch=2000, batch_size=20, validation_split=0.1,
          callbacks=[TensorBoard(log_dir=local_path + './tmp/log', histogram_freq=0, write_graph=True,
                                 write_images=True)])  # starts training


Using TensorFlow backend.


/Users/Zizy/anaconda/envs/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


NameError: name 'local_path' is not defined

In [22]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import TensorBoard

# This returns a tensor
inputs = Input(shape=(371,))

# a layer instance is callable on a tensor, and returns a tensor
# x = Dense(64, activation='relu')(inputs)
# x = Dense(32, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# predictions = Dense(2, activation='linear')(x)

model = load_model(local_path + 'best_timing.h5')
model.compile(optimizer=Adam(),
              loss='mse',
              )
model.summary()
model.fit(X, Y, nb_epoch=2000, batch_size=20, validation_split=0.1,
          callbacks=[TensorBoard(log_dir=local_path + './tmp/log', histogram_freq=0, write_graph=True,
                                 write_images=True)])  # starts training


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 371)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 128)               47616     
_________________________________________________________________
dense_41 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 258       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 3         
Total params: 64,389
Trainable params: 64,389
Non-trainable params: 0
_________________________________________________________________


/Users/Zizy/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 6077 samples, validate on 676 samples


Epoch 1/2000
  20/6077 [..............................] - ETA: 20s - loss: 6859.3955

 360/6077 [>.............................] - ETA: 1s - loss: 6668.9385 

...

1940/6077 [========>.....................] - ETA: 0s - loss: 10642.2830

2280/6077 [==========>...................] - ETA: 0s - loss: 10315.3258

2580/6077 [===========>..................] - ETA: 0s - loss: 9780.9877 

2980/6077 [=============>................] - ETA: 0s - loss: 9141.5013

KeyboardInterrupt: 

In [1]:
model.save(local_path+'best_timing2.h5')

NameError: name 'model' is not defined

In [101]:
X[0]

array([1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [38]:
model  = load_model(local_path+'best_timing.h5')


array([[1203.0356  ],
       [ -16.79428 ],
       [ 412.01486 ],
       [ 167.42834 ],
       [  58.739532],
       [ 235.51729 ],
       [ 186.0389  ],
       [  52.72004 ],
       [ 207.08897 ],
       [ 117.00125 ]], dtype=float32)

In [54]:
model.predict(X)[0:10]

array([[1203.0356  ],
       [ -16.79428 ],
       [ 412.01486 ],
       [ 167.42834 ],
       [  58.739532],
       [ 235.51729 ],
       [ 186.0389  ],
       [  52.72004 ],
       [ 207.08897 ],
       [ 117.00125 ]], dtype=float32)

In [52]:
t0 = np.array(get_params_by_name('t0', labs)).astype(float) / 10000
t1 = np.array(get_params_by_name('t1', labs)).astype(float) / 10000
np.reshape(t1, (len(t1), 1))

array([[ 1200.],
       [ 1650.],
       [ 1650.],
       ...,
       [11400.],
       [11400.],
       [13200.]])

In [49]:
convert_params_to_one_hot('p3', phonemes, labs).shape

(6753, 37)

In [53]:
X[0]

array([1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      